<a href="https://colab.research.google.com/github/Baaabaei/Baaabaei/blob/main/DataChunker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import json
import os

In [2]:
def clean_text(text):
    """پاکسازی متن از محتوای غیر ضروری"""
    # حذف فوتر و هدر
    text = re.sub(r'Copyright © .*?\n', '', text)
    text = re.sub(r'Disclaimer:.*?\n', '', text)

    # حذف خطوط خالی اضافی
    text = re.sub(r'\n\s*\n', '\n\n', text)

    # حذف کاراکترهای غیر متنی
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    return text.strip()



In [3]:
def create_chunks(input_file, output_dir, chunk_size=1000, chunk_overlap=100):
    """تبدیل متن به چانک‌های مناسب و ذخیره آنها"""

    # خواندن فایل ورودی
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # پاکسازی متن
    cleaned_text = clean_text(text)

    # تنظیم text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", "! ", "? ", ";", ":", " ", ""],
        length_function=len
    )

    # ایجاد چانک‌ها
    chunks = text_splitter.split_text(cleaned_text)

    # ایجاد دایرکتوری خروجی اگر وجود ندارد
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # ذخیره چانک‌ها در فایل‌های جداگانه و یک فایل JSON
    chunks_data = []
    for i, chunk in enumerate(chunks):
        # ذخیره هر چانک در یک فایل متنی
        chunk_file = os.path.join(output_dir, f'chunk_{i+1}.txt')
        with open(chunk_file, 'w', encoding='utf-8') as f:
            f.write(chunk)

        # اضافه کردن به لیست برای JSON
        chunks_data.append({
            'chunk_id': i+1,
            'content': chunk,
            'length': len(chunk)
        })

    # ذخیره همه چانک‌ها در یک فایل JSON
    json_file = os.path.join(output_dir, 'all_chunks.json')
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(chunks_data, f, ensure_ascii=False, indent=2)

    return len(chunks)

In [4]:


def analyze_chunks(output_dir):
    """تحلیل چانک‌های ایجاد شده"""
    with open(os.path.join(output_dir, 'all_chunks.json'), 'r', encoding='utf-8') as f:
        chunks = json.load(f)

    # محاسبه آمار
    lengths = [chunk['length'] for chunk in chunks]
    avg_length = sum(lengths) / len(lengths)
    max_length = max(lengths)
    min_length = min(lengths)

    analysis = {
        'total_chunks': len(chunks),
        'average_length': round(avg_length, 2),
        'max_length': max_length,
        'min_length': min_length
    }

    # ذخیره تحلیل
    analysis_file = os.path.join(output_dir, 'analysis.json')
    with open(analysis_file, 'w', encoding='utf-8') as f:
        json.dump(analysis, f, indent=2)

    return analysis

In [6]:


def main():
    # تنظیمات
    input_file = '/content/guidelines.txt'  # نام فایل ورودی را اینجا تغییر دهید
    output_dir = 'chunked_guidelines'
    chunk_size = 1000
    chunk_overlap = 100

    # ایجاد چانک‌ها
    num_chunks = create_chunks(input_file, output_dir, chunk_size, chunk_overlap)
    print(f'تعداد {num_chunks} چانک ایجاد شد.')

    # تحلیل نتایج
    analysis = analyze_chunks(output_dir)
    print('\nتحلیل چانک‌ها:')
    print(f'تعداد کل چانک‌ها: {analysis["total_chunks"]}')
    print(f'میانگین طول چانک‌ها: {analysis["average_length"]}')
    print(f'بیشترین طول: {analysis["max_length"]}')
    print(f'کمترین طول: {analysis["min_length"]}')

if __name__ == '__main__':
    main()

تعداد 883 چانک ایجاد شد.

تحلیل چانک‌ها:
تعداد کل چانک‌ها: 883
میانگین طول چانک‌ها: 691.98
بیشترین طول: 999
کمترین طول: 1


In [8]:
!zip -r /content/chunked_guidelines.zip /content/chunked_guidelines

  adding: content/chunked_guidelines/ (stored 0%)
  adding: content/chunked_guidelines/chunk_585.txt (deflated 46%)
  adding: content/chunked_guidelines/chunk_602.txt (deflated 43%)
  adding: content/chunked_guidelines/chunk_434.txt (deflated 46%)
  adding: content/chunked_guidelines/chunk_676.txt (deflated 27%)
  adding: content/chunked_guidelines/chunk_840.txt (deflated 48%)
  adding: content/chunked_guidelines/chunk_711.txt (deflated 58%)
  adding: content/chunked_guidelines/chunk_738.txt (deflated 41%)
  adding: content/chunked_guidelines/chunk_865.txt (deflated 47%)
  adding: content/chunked_guidelines/chunk_254.txt (deflated 50%)
  adding: content/chunked_guidelines/chunk_586.txt (stored 0%)
  adding: content/chunked_guidelines/chunk_469.txt (deflated 36%)
  adding: content/chunked_guidelines/chunk_108.txt (deflated 49%)
  adding: content/chunked_guidelines/chunk_700.txt (deflated 49%)
  adding: content/chunked_guidelines/chunk_171.txt (deflated 37%)
  adding: content/chunked_gui

In [9]:

 from google.colab import files
files.download("/content/chunked_guidelines.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>